In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/skylab_instagram_datathon_dataset.csv", sep=";")
df_original = df.copy(deep=True)
df.head()

,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
0,Weekly,2019-12-07,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,22066189.0,97.0,13.0,16239.0,6112205.0
1,Weekly,2021-01-30,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,24289963.0,54.0,2.0,13358.0,3189144.0
2,Weekly,2016-05-14,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,7137442.0,42.0,8.0,15894.0,2152955.0
3,Weekly,2019-04-20,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,18901125.0,73.0,9.0,26762.0,5224916.0
4,Weekly,2023-03-25,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,30251374.0,54.0,10.0,25681.0,5760987.0


<h3>Removing columns "period", "calculation_type" <h3\>

In [3]:
#removing "period", "calculation_type" since the number of followers, likes,... are independent of these cathegories(Comment out after first run)
df = df_original.copy(deep=True) 
df.drop(columns=["period", "calculation_type", "domicile_country_name","primary_exchange_name", "compset", "legal_entity_name","ultimate_parent_legal_entity_name"], inplace=True)
#df.drop(columns=["period", "calculation_type", "domicile_country_name","primary_exchange_name", "compset"], inplace=True)

df.sort_values(by=["period_end_date"], inplace=True, ascending=False)
display(df.head())
display(df.shape)

,period_end_date,compset_group,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
533918,2023-09-16,Luxury & Premium & Mainstream,Moncler,5232719.0,6.0,4.0,361.0,63022.0
508933,2023-09-16,Luxury & Premium & Mainstream,IWC Schaffhausen,2426903.0,84.0,57.0,1094.0,178262.0
676605,2023-09-16,Apparel Retail,Pull&Bear,7842273.0,18.0,21.0,777.0,170730.0
125631,2023-09-16,Luxury & Premium & Mainstream,Jaeger-LeCoultre,1346806.0,29.0,12.0,832.0,155136.0
96155,2023-09-16,Luxury & Premium & Mainstream,Armani,27257224.0,140.0,25.0,10208.0,782899.0


(704313, 8)

In [4]:
#Assumption: A brand will not be in different compset_groups over time!
df.drop_duplicates(inplace=True)

#now after removing duplicte rows, we can remove compset_group column
df.drop(columns=["compset_group"], inplace=True)

df.sort_values(by=["business_entity_doing_business_as_name", "period_end_date"], inplace=True, ascending=False)
df.reset_index(drop=True, inplace=True)
display(df.shape)
display(df.head())

(325732, 7)

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
0,2023-09-16,shopDisney,4239207.0,244.0,72.0,3908.0,330793.0
1,2023-09-09,shopDisney,4233764.0,259.0,56.0,3700.0,335231.0
2,2023-09-02,shopDisney,4228605.0,268.0,61.0,3920.0,341795.0
3,2023-08-26,shopDisney,4224473.0,265.0,61.0,3494.0,316126.0
4,2023-08-19,shopDisney,4221812.0,266.0,52.0,3099.0,266166.0


In [5]:
# #needed to evaluate "All Brands" brand data

# df_tests = df[df["business_entity_doing_business_as_name"] =="All Brands"]
# df_tests = df_tests[df_tests["compset_group"] =="Luxury & Premium & Mainstream"]
# df_tests.shape

# #get counts of unique values in each column
# for col in df_tests.columns:
#     print("Sum per column: ", df_tests[col].value_counts().sum())

#     print(col)
#     display(df_tests[col].value_counts())
#     print(len(df_tests[col].value_counts()))
#     print("\n")

# remove all rows with "All Brands" in business_entity_doing_business_as_name
df = df[df["business_entity_doing_business_as_name"] != "All Brands"]
df.reset_index(drop=True, inplace=True)



In [6]:
#df.pivot(index ="period_end_date", 
        #columns= ["compset_group", "business_entity_doing_business_as_name", "legal_entity_name", "ultimate_parent_legal_entity_name"],
        #values=["followers", "pictures", "videos", "comments", "likes"]).head()

#check for nan values in the dataset
nan_rows = df.isna().sum()
display(nan_rows)


#get rows with nan values
df[df.isna().any(axis=1)]


# columns = df.columns
# print(columns)
# df_sum = 0

# for i in columns:
    
#     print("\n")
#     print("-----------------------------------------------------------------------------")
#     print(i, "--> has length:", len(df[i].unique()))
#     df_i = df[i].value_counts().reset_index()
#     df_i.columns = [i, "count"]
#     display(df_i)
#     df_sum = df_i["count"].sum()
#     print("Sum of count:", df_sum)
#     df_sum = 0
#     print("-----------------------------------------------------------------------------")
#     print("\n")




period_end_date                               0
business_entity_doing_business_as_name        0
followers                                 61727
pictures                                   3864
videos                                     9602
comments                                   4082
likes                                      3752
dtype: int64

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
345,2022-04-23,iRobot,NaN,136.0,28.0,308.0,4836.0
346,2022-04-16,iRobot,NaN,132.0,27.0,5367.0,6450.0
347,2022-04-09,iRobot,NaN,132.0,30.0,5407.0,6378.0
348,2022-04-02,iRobot,NaN,125.0,41.0,5408.0,6003.0
349,2022-03-26,iRobot,NaN,120.0,41.0,5441.0,6365.0
...,...,...,...,...,...,...,...
298035,2017-06-03,24S,NaN,24.0,3.0,109.0,3922.0
298036,2017-05-27,24S,NaN,6.0,3.0,57.0,1765.0
298037,2017-05-20,24S,NaN,6.0,3.0,57.0,1765.0
298038,2017-05-13,24S,NaN,6.0,3.0,57.0,1765.0


<h3> Using Timedelta instead of Absolute datetime and ordering by datetime <h3>

In [7]:
#taking timedelta from first available end date instead of absolute date (COMMENT OUT AFTER FIRST RUN)
#df['period_end_date'] = pd.to_datetime(df['period_end_date'])
#min_date = df['period_end_date'].min()
#df['period_end_date'] = df['period_end_date'] - min_date
#order by period_end_date
df.sort_values(by=["business_entity_doing_business_as_name", "period_end_date"], inplace=True, ascending=False)


<h3> where do we have missing values?<h3\>

In [8]:
for name in df.columns:
    if df[df[name].isnull()].shape[0] > 0:
        print("number of missing values for ",name, ": ",df[df[name].isnull()].shape[0])

number of missing values for  followers :  61727
number of missing values for  pictures :  3864
number of missing values for  videos :  9602
number of missing values for  comments :  4082
number of missing values for  likes :  3752


<h3>Investigation of Domicile Country Name<h3\>

In [9]:
#COMMENT OUT AFTER FIRST RUN
#nan_domicile = df[df['domicile_country_name'].isnull()]
#nan_domicile["business_entity_doing_business_as_name"].unique().size
#check if brand has non-null domicile for each brand
#for brand in nan_domicile["business_entity_doing_business_as_name"].unique():
    #known_brands = []
    #check if brand has non-null domicile
    #if df[df["business_entity_doing_business_as_name"] == brand]["domicile_country_name"].notnull().any():
        #known_brands.append(brand)
        #print(brand)
#the result is empty, so we can't get the domicile from another row
#remove domicile_country_name column
#df.drop(columns=["domicile_country_name"], inplace=True)

In [10]:
cathegories = ["followers", "likes", "comments", "videos", "pictures"]
for name in cathegories:
    print("number of missing values for ",name, ": ",df[df[name].isnull()].shape[0] + df[df[name] == 0].shape[0])
    print("number of zeros for ",name, ": ",df[df[name] == 0].shape[0])
    #ratio of zeros + nan to total
    print("ratio of zeros + nan to total for ",name, ": ",(df[df[name].isnull()].shape[0] + df[df[name] == 0].shape[0])/df.shape[0])
    


number of missing values for  followers :  61740
number of zeros for  followers :  13
ratio of zeros + nan to total for  followers :  0.20715340222788886
number of missing values for  likes :  9099
number of zeros for  likes :  5347
ratio of zeros + nan to total for  likes :  0.03052945913300228
number of missing values for  comments :  10138
number of zeros for  comments :  6056
ratio of zeros + nan to total for  comments :  0.03401556838008321
number of missing values for  videos :  51186
number of zeros for  videos :  41584
ratio of zeros + nan to total for  videos :  0.17174204804724197
number of missing values for  pictures :  10827
number of zeros for  pictures :  6963
ratio of zeros + nan to total for  pictures :  0.03632733861226681


In [11]:
#fill nan with zeros
for name in cathegories:
    df[name].fillna(0, inplace=True)

In [12]:
# brands_missing_values = pd.DataFrame(columns=["brand", "cathegory", "ratio"])

# for name in df["business_entity_doing_business_as_name"].unique():
#     for cathegory in cathegories:
#         nr_missing_values = df[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0))].shape[0]
#         ratio = nr_missing_values / df[df["business_entity_doing_business_as_name"] == name].shape[0]
#         if ratio > 0.3:
#             brands_missing_values.loc[len(brands_missing_values.index)] = [name, cathegory, ratio]
#             #print("brand: ",name, " cathegory: ",cathegory, " ratio: ",ratio)
#brands_missing_values.to_csv("data/brands_missing_values.csv")

In [13]:
#filling missing values with the mean of the brand 


# for name in df["business_entity_doing_business_as_name"].unique():
#     for cathegory in cathegories:
#         #if there are zeros or nan values for a brand, fill them with the mean of the brand
#         if df[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0))].shape[0] > 0:
#             mean = df[df["business_entity_doing_business_as_name"] == name][cathegory].mean()
#             df.loc[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0)), cathegory] = mean


In [14]:
#df.pivot(index ="period_end_date", 
        #columns= ["compset_group", "business_entity_doing_business_as_name", "legal_entity_name", "ultimate_parent_legal_entity_name"],
        #values=["followers", "pictures", "videos", "comments", "likes"]).head()

#check for nan values in the dataset
nan_rows = df.isna().sum()
display(nan_rows)


#get rows with nan values
df[df.isna().any(axis=1)]


columns = df.columns
print(columns)
df_sum = 0

for i in columns:
    
    print("\n")
    print("-----------------------------------------------------------------------------")
    print(i, "--> has length:", len(df[i].unique()))
    df_i = df[i].value_counts().reset_index()
    df_i.columns = [i, "count"]
    display(df_i)
    df_sum = df_i["count"].sum()
    print("Sum of count:", df_sum)
    df_sum = 0
    print("-----------------------------------------------------------------------------")
    print("\n")




period_end_date                           0
business_entity_doing_business_as_name    0
followers                                 0
pictures                                  0
videos                                    0
comments                                  0
likes                                     0
dtype: int64

Index(['period_end_date', 'business_entity_doing_business_as_name',
       'followers', 'pictures', 'videos', 'comments', 'likes'],
      dtype='object')


-----------------------------------------------------------------------------
period_end_date --> has length: 455


,period_end_date,count
0,2023-09-16,705
1,2023-06-10,705
2,2023-09-09,705
3,2023-03-25,705
4,2023-04-01,705
...,...,...
450,2015-01-31,454
451,2015-01-24,454
452,2015-01-17,454
453,2015-01-10,453


Sum of count: 298040
-----------------------------------------------------------------------------




-----------------------------------------------------------------------------
business_entity_doing_business_as_name --> has length: 705


,business_entity_doing_business_as_name,count
0,Loewe,455
1,Clinique,455
2,Marshall's,455
3,Mary Kay,455
4,Massimo Dutti,455
...,...,...
700,Sculptra,129
701,Temu,55
702,Finding Unicorn,46
703,Pop Mart,46


...

Sum of count: 298040
-----------------------------------------------------------------------------




-----------------------------------------------------------------------------
comments --> has length: 38638


,comments,count
0,0.0,10138
1,3.0,569
2,7.0,511
3,5.0,510
4,11.0,508
...,...,...
38633,12954.0,1
38634,12939.0,1
38635,12384.0,1
38636,10659.0,1


Sum of count: 298040
-----------------------------------------------------------------------------




-----------------------------------------------------------------------------
likes --> has length: 177461


,likes,count
0,0.0,9099
1,79.0,35
2,194.0,33
3,228.0,33
4,386.0,33
...,...,...
177456,742374.0,1
177457,476925.0,1
177458,393788.0,1
177459,337186.0,1


Sum of count: 298040
-----------------------------------------------------------------------------




In [15]:
display(df.head(10))
print(df.shape)

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
0,2023-09-16,shopDisney,4239207.0,244.0,72.0,3908.0,330793.0
1,2023-09-09,shopDisney,4233764.0,259.0,56.0,3700.0,335231.0
2,2023-09-02,shopDisney,4228605.0,268.0,61.0,3920.0,341795.0
3,2023-08-26,shopDisney,4224473.0,265.0,61.0,3494.0,316126.0
4,2023-08-19,shopDisney,4221812.0,266.0,52.0,3099.0,266166.0
5,2023-08-12,shopDisney,4216486.0,258.0,57.0,3201.0,292773.0
6,2023-08-05,shopDisney,4211757.0,260.0,58.0,3329.0,323431.0
7,2023-07-29,shopDisney,4205824.0,274.0,42.0,3147.0,365995.0
8,2023-07-22,shopDisney,4204007.0,268.0,45.0,3671.0,405953.0
9,2023-07-15,shopDisney,4200757.0,264.0,39.0,3756.0,420512.0


(298040, 7)


In [16]:
display(df.tail(10))
print(df.shape)

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
298030,2017-07-08,24S,0.0,65.0,16.0,193.0,16537.0
298031,2017-07-01,24S,0.0,47.0,25.0,195.0,14943.0
298032,2017-06-24,24S,0.0,44.0,25.0,201.0,12533.0
298033,2017-06-17,24S,0.0,33.0,18.0,150.0,9352.0
298034,2017-06-10,24S,0.0,21.0,9.0,89.0,4782.0
298035,2017-06-03,24S,0.0,24.0,3.0,109.0,3922.0
298036,2017-05-27,24S,0.0,6.0,3.0,57.0,1765.0
298037,2017-05-20,24S,0.0,6.0,3.0,57.0,1765.0
298038,2017-05-13,24S,0.0,6.0,3.0,57.0,1765.0
298039,2017-05-06,24S,0.0,0.0,0.0,0.0,0.0


(298040, 7)


In [21]:
#save the cleaned data
df.to_csv("data/cleaned_data.csv", index=False)
df.shape

(298040, 7)

In [20]:
pd.read_csv("data/cleaned_data.csv").head()

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
0,2023-09-16,shopDisney,4239207.0,244.0,72.0,3908.0,330793.0
1,2023-09-09,shopDisney,4233764.0,259.0,56.0,3700.0,335231.0
2,2023-09-02,shopDisney,4228605.0,268.0,61.0,3920.0,341795.0
3,2023-08-26,shopDisney,4224473.0,265.0,61.0,3494.0,316126.0
4,2023-08-19,shopDisney,4221812.0,266.0,52.0,3099.0,266166.0


In [19]:
pd.read_csv("data/cleaned_data.csv").tail()

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
298035,2017-06-03,24S,0.0,24.0,3.0,109.0,3922.0
298036,2017-05-27,24S,0.0,6.0,3.0,57.0,1765.0
298037,2017-05-20,24S,0.0,6.0,3.0,57.0,1765.0
298038,2017-05-13,24S,0.0,6.0,3.0,57.0,1765.0
298039,2017-05-06,24S,0.0,0.0,0.0,0.0,0.0
